# Python 开发环境搭建

## 下载MiniConda

Conda的地址：https://conda.io/en/latest/index.html

```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
```

## 安装MiniConda

下载下来的本身就是一个可执行的安装脚本，我们可以查看它的安装选项

```
$ chmod +x Miniconda3-latest-Linux-x86_64.sh
$ ./Miniconda3-latest-Linux-x86_64.sh -h
usage: ./Miniconda3-latest-Linux-x86_64.sh [options]

Installs Miniconda3 py38_4.9.2

-b run install in batch mode (without manual intervention), it is expected the license terms are agreed upon
-f no error if install prefix already exists
-h print this help message and exit
-p PREFIX install prefix, defaults to $HOME/miniconda3, must not contain spaces.
-s skip running pre/post-link/install scripts
-u update an existing installation
-t run package tests after installation (may install conda-build)
```

一般来说，直接使用-b选项即可，安装的位置位于`$HOME/miniconda3`

**设置环境变量**

```
CONDA_PATH=$HOME/miniconda3/bin
export PATH=$CONDA_PATH:$PATH
```
设置后需要重启Bash

**初始化Conda**

conda init bash

**配置清华镜像**

在IDC尝试换清华源后，出现连接超时的问题，直接使用官方源速度还可以，只有pytorch(1.28G)慢一些

参考链接：https://mirrors.tuna.tsinghua.edu.cn/help/anaconda/

将以下内容写入$HOME/.condarc文件

```
channels:
  - defaults
show_channel_urls: true
default_channels:
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/r
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/msys2
custom_channels:
  conda-forge: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  msys2: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  bioconda: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  menpo: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  pytorch: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  simpleitk: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
```

## 创建一个python3.8的虚拟开发环境

```
conda create -n pyml python=3.8 -y
```

其中`-n pyml`等价于`–name pyml`，指定了创建的虚拟开发空间为`pyml`，`python=3.8`指定了这个空间初始化安装的软件包及其版本，-`y`说明了跳过提示。

激活虚拟环境：`conda activate pyml` 效果如下:

```
(base) [yangyansheng@stc-10-211-19-15 ~]$ conda activate pyml
(pyml) [yangyansheng@stc-10-211-19-15 ~]$
```

## 安装深度学习相关的工具包

- 安装Pytorch：`conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c conda-forge`
- 安装《动态学习深度学习》课程配套代码包d2l：　`pip install d2l`
- 安装jupyterlab：`conda install -c conda-forge jupyterlab` 或者通过`pip install jupyterlab`

## 配置Jupyter Server

默认情况下Jupyter Server只能运行在本地地址：`127.0.0.1(localhost):8888`上。

**生成配置文件**

生成jupyter server的配置文件：`jupyter server --generate-config`
生成的配置文件默认路径是：`$HOME/.jupyter/jupyter_server_config.py`

**配置密码**

默认情况下，JupyterServer都是通过一个token来访问的，在我们第一次通过Web页面登录时，是可以选择通过页面来设置密码的。

也可以选择通过命令行来设置密码：`jupyter server password`

```
jupyter server password
Enter password: ****
Verify password: ****
[JupyterPasswordApp] Wrote hashed password to $HOME/.jupyter/jupyter_server_config.json
```

我们可以看到jupyter_server_config.json中多了一些配置
```
{
"ServerApp": {
"password": "argon2:$argon2id$v=19$m=10240,t=10,p=8$KPPAF2qrPp*****************r+I6reVXTfMWdtA"
}
```

另外一种配置密码的方法
先通过`notebook.auth.security.passwd()`来生成一个密码的Hash字符串
```
from notebook.auth.security import passwd

passwd()

Enter password: *****

Verify password: *****

"argon2:$argon2id$v=19$m=10240,t=10,p=8$KPPAF2qrPp*****************r+I6reVXTfMWdtA"
```

会生成和上一种方法一致的密码的Hash码。

然后通过配置`jupyter_server_config.py`中的`c.ServerApp.password`变量：
```
c.ServerApp.password = u'argon2:$argon2id$v=19$m=10240,t=10,p=8$KPPAF2qrPp*****************r+I6reVXTfMWdtA'
```

值得注意的是`jupyter_server_config.json`中配置项的优先级要高于`jupyter_server_config.py`

**远程访问Jupyter Server**

在远程服务器上运行Jupyter Server，下面以Jupyter lab为例

```
jupyter lab --port=8889 --no-browser
```

在本地机器上设置SSL代码，将本地8889端口映射到远程服务器的端口：

```
ssh L 8889:localhost:8889 user@remote-server
```

然后我们就可以在本地的浏览器中输出`localhost:8889`来查看jupyter lab了

有没有办法不通过SSL代理来直接访问romote-ip:8889呢？应该也是可行的，之前是通过设置--ip='0.0.0.0'就可以了，但没有验证work，可能是同时需要证书支持。

参考链接：https://jupyter-server.readthedocs.io/en/latest/operators/public-server.html

Jupyter Notebook和Jupyter Server的配置不是一同个，https://jupyter-server.readthedocs.io/en/latest/operators/migrate-from-nbserver.html